# Exercises 

These exercises provide a short look at the "numerical" and "computational" challenges of density-functional theory (DFT). For the work here we will use the [density-functional toolkit](https://dftk.org) (DFTK), a small Julia code for plane-wave DFT simulations. 

Some modification of Julia code is required to do the exercises, but apart from that you can do the analysis of your numerical results in any form you want. But naturally this would be a good opportunity to learn some Julia ;).
 
Some help in case you get stuck and feel you would like to learn more:
- https://michael-herbst.com/learn-julia: One-day Julia introductory course
- https://julialang.org/learning/: Various learning resources for Julia
- https://docs.julialang.org/: Julia documentation

## Installing Julia

1. Download Julia from https://julialang.org/downloads/, choose the tarball fitting your operating system. You neet to use at least **Julia 1.6**.
1. Unpack / install the tarball and start julia. More detailed [installation instructions](https://julialang.org/downloads/platform.html).
1. You should see a `julia>` prompt. In this prompt install the [IJulia.jl](https://github.com/JuliaLang/IJulia.jl) package (Jupyter Julia kernel):
   ```julia
   using Pkg
   Pkg.add("IJulia")
   ```
1. If you don't yet have Jupyter installed, then the easiest is to do it from Julia.
   Again from the `julia>` prompt, run:
   ```julia
   using IJulia
   notebook()
   ```
   If you do already have Jupyter (and it is part of your PATH) than this should start your browser.
   If this does not work, see [the IJulia documentation](https://julialang.github.io/IJulia.jl/stable/manual/installation/) for more help and details.
5. Start this Jupyter notebook and execute it in your browser.

## Installing ASE
This and the following assumes you have installed Julia as detailed above and you are running
this notebook from within your own browser.

For exercise 2 we will need one python package called [ASE](https://wiki.fysik.dtu.dk/ase/). For simplicity we will install this inside Julia's own python environment, which works by running the following cells:

In [ ]:
using Pkg
ENV["PYTHON"] = ""
Pkg.add("PyCall")
Pkg.add("Conda")
Pkg.build("PyCall")
using Conda
Conda.add("ase", channel="conda-forge")

## Installing DFTK 

To install DFTK and the Julia packages required for this exercise run:

In [ ]:
using Pkg
Pkg.add("DFTK")
Pkg.add("Plots")
Pkg.add("PyCall")

## Setting up DFTK 

Do this every time you start the notebook to do some work:

In [ ]:
# Setup DFTK
using DFTK
setup_threading()

## Exercise 1: Why do we need pseudopotentials? (4 points)

It was discussed in the lecture that plane-wave DFT codes usually employ so-called pseudopotentials (PSP) to model the interaction of electrons and nuclei. Replacing the true Coulombic interaction by a PSP is an approximation. Multiple types of PSPs exist and from a mathematical perspective little is understood about the errors introduced by PSPs. However, from a physical point of view using PSPs is reasonable as it only effects the electron density close to the nuclei, which for most phaenomena only plays a minor role. On the upside PSPs turn out to make plane-wave calculations much more feasible.

This aspect we will investigate numerically in this exercise. Our goal will be to determine the energy of a single neon atom using the so-called LDA approximation up to an absolute error of `0.1`. We will do this by an explicit convergence study, i.e. by increasing the basis size and improving the numerics until we are sure we have found the energy to this tolerance.

Our computational setup is simple: We will put a single neon atom into a cubic box of length $a$. As DFTK uses periodic boundary conditions, this atom interacts spuriously with the neighbouring cells, introducing an error to our calculation. As $a \to \infty$ this error disappears, which is one of the convergence parameters we will study. The other is the $E_\text{cut}$ parameter discussed in the lecture, which determines the basis set size. Again calculations get better for $E_\text{cut} \to \infty$. In the language of DFTK the calculation we want to perform is:

In [ ]:
using LinearAlgebra

# Numerical parameters
#
a    = 5   # Box size 
Ecut = 30  # Plane-wave basis cutoff

# Use a HGH pseudopotential:
Ne = ElementPsp(:Ne, psp=load_psp("hgh/lda/ne-q8"))

# Use the true Coulomb interaction:
# Ne = ElementCoulomb(:Ne)
#
# End numerical parameters 

# Setup Cubic box and place neon atom in the middle
lattice = a * Matrix(I, 3, 3)
atoms = [Ne => [[0.5, 0.5, 0.5]]]

# Use LDA model, discretise and run SCF algorithm
model  = model_LDA(lattice, atoms)
basis  = PlaneWaveBasis(model, Ecut, kgrid=[1, 1, 1])
scfres = self_consistent_field(basis)
Etot   = scfres.energies.total

# Print total energy
println("\nTotal DFT energy is $Etot")

**Warning:** DFT calculations can be both time and memory consuming. When doing these convergence studies therefore start with small values for `a` and `Ecut` (like the ones shown here) and increase slowly, especially if you are running these on a laptop with 8GB of main memory or less. You are not expected to re-compute the provided reference results.

**(a) Convergence in the box size.**

(i) First stick with the pseudpotential (PSP) version of the neon atom (`ElementPsp`). At the level of `Ecut = 30` converge the DFT energy to an error of `0.1` by increasing the value of `a`. The easiest way to do this is to run multiple calculations for different values of `a` and plot the error against a reference on a semilog scale (see the [Plots.jl](http://docs.juliaplots.org/latest/generated/gr/) documentation if you want to do this in Julia). A good reference is at `a = 50`, where `Etot = -33.0054`.

(ii) Next we compare a PSP calculation with a full Coulomb potential calculation, also called all-electron calculation. Pick a small value for `a` and run both the PSP calculation and the all-electron one (`ElementCoulomb`). What differences in runtime and the obtained value do you observe? Suggest an explanaition keeping in mind that part of the idea of the PSP is to avoid the explicit treatment of the core electrons.

Regarding the timings you might find the `@time` Julia macro handy, which measures the time of a following Julia command, e.g.

In [ ]:
@time sleep(2)

(iii) Is the value for `a` you found in (i) sufficient to converge the calcuation to an absolute energy error of `0.1` in case `ElementCoulomb` is employed? Stick with `Ecut = 30` in this exercise. 
*Hint*: Keep the timings you found in (ii) in mind and try to do as few calculations with `ElementCoulomb` as possible.

**(b) Convergence in the basis cutoff.**

In this part of the exercise we fix `a = 10` and consider the convergence in `Ecut`.

(i) For the PSP model, converge the energy to an error of `0.1`. Notice that for `a = 10` and `Ecut = 400` the DFT energy is `-34.8533`.

(ii) Repeat the exercise for the all-electron case. Here the reference result at `Ecut = 400` is `-121.0346`.

(iii) Suggest values for `a` and `Ecut` that overall achieve an absolute energy error of `0.1` if the PSP is employed. State the runtimes of a DFT calculation with these numerical parameters with and without PSP approximation.

## Exercise 2: SCF algorithms to solve DFT (3 points)

The self-consistent field procedure required to solve the DFT problem can be written as a fixed-point problem
$$ F(\rho) = \rho $$
where $F$ represents the basic SCF step. That is the construction of the Kohn-Sham Hamiltonian $H(\rho)$ given the density $\rho$, followed its diagonalisation to obtain its eigenpairs $(\varepsilon_{k i}, \psi_{ki})$
and from these a new density
$$ \rho(r) = \sum_{k\in\text{BZ}} \sum_i f_{\varepsilon_F}(\varepsilon_{ki}) \, \psi_{ki}(r) \, \psi_{ki}^\ast(r)$$
with the Fermi level $\varepsilon_F$ chosen to conserve the number of electrons.

In this exercise we will not be concerned with $F$, much rather we take this function as "granted" (i.e. delivered by DFTK). Instead what we will investigate are multiple algorthms to find the fixed-point density satisfying $F(\rho) = \rho $. Our setting will be the following example script:

In [ ]:
using DFTK
using PyCall
read = pyimport("ase.io").read

# Load base system (Silicon)
system = read("silicon.json")

# Parameters
#
# Make problem harder by repeating the system in x direction ...
N = 1  # increase later to 2, 4, 8
system = system * (N, 1, 1)
#
# End parameters 

# Setup model
lattice = load_lattice(system)
atoms   = load_atoms(system)
atoms   = [ElementPsp(el.symbol, psp=load_psp(el.symbol, functional="pbe")) => position
           for (el, position) in atoms]
model = model_LDA(lattice, atoms)

# Setup basis
Ecut  = 5
basis = PlaneWaveBasis(model, Ecut; kgrid=[1, 1, 1]);

**(a) Fixed-point iterations.** The easiest way to solve $F(\rho) = \rho$ are fixed-point iterations, i.e.
$$ \rho_{n+1} = F(\rho_n), $$
starting from a hopefully good initial guess $\rho_0$. DFTK automatically provides a reasonable
guess density, such that we only need to take care of the iterations themselves.
In the language of DFTK this algorithm is written as:

In [ ]:
function fixed_point_iteration(F, ρ₀, maxiter; tol)
    # F:        The SCF step function
    # ρ₀:       The initial guess density
    # maxiter:  The maximal number of iterations to be performed
    # tol:      The selected convergence tolerance
    
    ρ  = ρ₀
    Fρ = F(ρ)
    for n = 1:maxiter 
        # If change less than tolerance, break iterations:
        if norm(Fρ - ρ) < tol
            break
        end
        ρ  = Fρ
        Fρ = F(ρ)
    end
    
    # Return some stuff DFTK needs ...
    (fixpoint=ρ, converged=norm(Fρ-ρ) < tol)
end;

# use this algorithm inside DFTK's SCF for solving the silicon problem
# (the other parameters are needed to overwrite some DFTK defaults we don't want to use yet).
self_consistent_field(basis; solver=fixed_point_iteration, α=1.0, maxiter=40);

As can be observed this algorithm is not very good and in fact even fails to converge albeit we are only looking at a very simple system.

This is a known limitation of this algorithm, which is why it is not used in practice. Instead one introduces a so-called damping parameter $\alpha$, which is given a value between $0$ and $1$. One now iterates as follows:
$$ \rho_{n+1} = \rho_{n} + \alpha (F(\rho_n) - \rho_n) $$
In other words the update $F(\rho_n) - \rho_n$ proposed in the $n$-th SCF step is not fully taken, but scaled-down by the damping $\alpha$.

Modify `fixed_point_iteration` such that it supports this *damped* fixed-point iteration. Try different values for $\alpha$ between $0$ and $1$ and estimate roughly the $\alpha$ which gives fastest convergence. For which $\alpha$ do you observe no convergence at all?

**Note:** The observations you make here are general. One can proove that every SCF converges (locally) if a small enough $\alpha > 0$ is chosen.

**(b) Anderson acceleration.** The `fixed_point_iteration` function above (with the damping extension) actually already covers the main gist of the DFT algorithms used in practice. One additional idea to make things converge faster is Anderson acceleration, where not only $\rho_n$ and $F(\rho_n)$, but also older iterates are used to propose the next density. For Anderson one exploits that the update $R(\rho) = F(\rho) - \rho$ is also the residual of the fixed-point problem $F(\rho) = \rho$, i.e. how far away we are from the fixed-point density. A good next density $\rho_{n+1}$ therefore should be found by minimising an approximation for $R(\rho_{n+1})$. Assuming the SCF was linear in the density (which it is not), a good idea is to find a linear
combination of residuals
$$\sum_i \beta_i R(\rho_i)$$
which has the smallest possible norm and to use these coefficients $\beta_i$ to extrapolate the next
density
$$ \rho_{n+1} =  \sum_i \beta_i (\rho_i + \alpha R(\rho_i)) $$
where you notice the "standard" damped fixed-point iteration in the summed terms.

This is the key idea of Anderson, which is the default SCF algorithm in DFTK (i.e. used whenever `solver` is not specified). When using this default solver in DFTK the damping $\alpha$ is specified by the same-named parameter of `self_consistent_field`, i.e.
```julia
self_consistent_field(basis; α=0.8);
```
runs Anderson with a damping $\alpha = 0.8$.

(i) Based on DFTK's Anderson implementation verify (by making a few experiments) that the algorithm converges for any $0 < \alpha \leq 2$. State the number of iterations and runtimes you observe.

(ii) Pick $\alpha = 0.6$ and make the problem harder by increasing `N` (e.g. `2`, `4`, `8`, `16`). Can you make Anderson fail to converge? What do you notice in terms of the number of iterations and runtimes?

**(c) Mixing methods.** Anderson allows us to push the boundary for SCF methods, but for larger or more challenging systems it is not fully sufficient. The next ingredient for a stable SCF procedure is based on the insight that the convergence properties of an SCF provably depend on the dielectric properties of materials, which is simulated. Amongst others this is to say that insulators (like glass), semiconductors (like silicon) or metals (like aluminium) have rather differing SCF behaviours. Therefore an ideal SCF procedure for each material is slightly different, an aspect we have not yet considered so far.

Typically one takes care of this by donig *preconditioned* damped fixed-point iterations, i.e. in the framework without Anderson acceleration of (a) one would iterate
$$ \rho_{n+1} = \rho_{n} + \alpha P^{-1} (F(\rho_n) - \rho_n) $$
where $P^{-1}$ is a preconditioner that uses an approximations for the dielectric properties of the material in order to improve convergence. Notice, that this idea carries forward to the Anderson-accelerated setting (just use  $R(\rho) = P^{-1} (F(\rho_n) - \rho_n)$, basically).

Finding a good preconditioner $P$ is not always easy and for some cases satisfactory options are not yet known. For our silicon case, however, we are lucky. The `DielectricMixing` implemented in DFTK provides a good $P$ for silicon. You might wonder about the term *mixing*. In an interdisciplinary community like DFT, different scientists use different vocabulary and "mixing" is the physicists' term used for preconditioning.

To use `DielectricMixing` with DFTK run the SCF as follows
```julia
self_consistent_field(basis; α=0.8, mixing=DielectricMixing());
```
Try this setup for different values of `N` and check the number of iterations needed. Other mixings DFTK has to offer are `KerkerMixing` (best for metals) or `LdosMixing` (best for metal-insulator-mixtures). Try them as well and summarise your findings.
You should notice that choosing a preconditioner matching the material under study aids a fast SCF convergence, but that sometimes being off does not seem to do much harm for our case. For larger values of `N` (beyond what you can probably effort on your laptop) this is no longer true and one needs to be very careful in selecting the right preconditioner. See for example the investigation in [this paper](https://michael-herbst.com/publications/2020.09.03_ldos_preconditioning.pdf). 

**Outlook:** Now you have at least heard about all the main numerical ingredients how DFT is solved in practice. Understanding the convergence properties of the SCFs and thus suggesting appropriate preconditioners $P$ would now of course be the next step, but unfortunately is beyond this course. Still it is a fascinating subject and I will talk some more about this at a (recorded) workshop at [JuliaCon 2021](https://pretalx.com/juliacon2021/featured) ...

## Excercise 3: Why iron is magnetic and silicon is not (3 points)

Only few compounds exhibit a natural permanent magnetism. One of these is iron, while most (like silicon) are not magnetic. This exercise tries to provide a little insight how one could understand, using DFT simulations, why this is the case.

The key assumption in this exercise will be that DFT is a realistic model and that the SCF therefore finds a good approximation to the true ground state of a compound. If this ground state turns out to be magnetic, the compound should therefore feature a permanant magnetism.

We use this computational setup to simulate silicon:

In [ ]:
using DFTK

a = 10.26
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]]

# Guess some inital magnetic moments
# (Need to be != 0 otherwise SCF makes the assumption that the ground state is not magnetic
#  to gain some performance ...)
magnetic_moments = [Si => [2, 2]]

model  = model_LDA(lattice, atoms; temperature=0.01, magnetic_moments=magnetic_moments)
Ecut   = 13
basis  = PlaneWaveBasis(model, Ecut; kgrid=[2, 2, 2]);
ρ0     = guess_density(basis, magnetic_moments)
scfres = self_consistent_field(basis, ρ=ρ0, mixing=KerkerMixing());
scfres.energies.total

Even though we have forced some magnetism into the initial density guess, this magnetisation (indicated by `Magnet`) disappears as the SCF converges. Therefore silicon appears to have a non-magnetic ground state.

(i) Repeat the calculation for iron. In this case the system setup is
```julia
a = 5.42352  # iron lattice constant in bohr
lattice = a / 2 * [[-1  1  1];
                   [ 1 -1  1];
                   [ 1  1 -1]]
Fe = ElementPsp(:Fe, psp=load_psp("hgh/lda/Fe-q8.hgh"))
atoms = [Fe => [zeros(3)]];
magnetic_moments = [Fe => [3]]
```
otherwise use the same settings as in the silicon calculation. Based on this calculation, what would you conclude?

(ii) As it turns out too small basis sets and Brilouin-zone sampling (too small `kgrid`s) are not able to support magnetic ground states. Repeat both the silicon as well as the iron calculations for different values of `Ecut` and `kgrid` (i.e. `[1,1,1]`,`[3,3,3]`, `[4,4,4]` etc.), where in both cases larger means better. Play with these parameters to determine the first two digits of the ground state energy of silicon and iron. Based on these numerical parameters what do you conclude now?

(iii) To show that a non-magnetic iron structure is actually higher in energy re-run the iron calculation with the `Ecut` and `kgrid` parameters determined in (ii), but this time set `magnetic_moments = [Fe ==> [0]]`. This enforces the SCF to converge to a non-magnetic ground state. This is why the magnetisation `Magnet` is no longer printed ... it is `0` by assumption. What is the energy difference between this non-magnetic iron ground state and the ground state you determined in (ii)?